# Historical Constituent of S&P500

Using Python by web scraping Wikipedia as it provides more historical data. [Reference](https://analyzingalpha.com/sp500-historical-components-and-changes)



In [1]:
import pandas as pd
import requests
from datetime import datetime
import numpy as np

In [2]:
headers = {
    'Accept': '*/*',
    'Accept-Language': 'en-US,en;q=0.8',
    'Cache-Control': 'max-age=0',
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/48.0.2564.116 Safari/537.36',
    'Connection': 'keep-alive',
    'Referer': 'http://www.baidu.com/'
}
# 设置本地代理
proxies  =  {
   "http" :  "http://127.0.0.1:1080" ,
   "https" :  "http://127.0.0.1:1080" ,
}

url = 'https://en.wikipedia.org/wiki/List_of_S&P_500_companies'
html = requests.get(url, headers=headers, proxies=proxies).content
data = pd.read_html(html)

In [50]:
sp500 = data[0].iloc[1:,[0,1,6,7]]
columns = ['added_ticker', 'name', 'date', 'cik']
sp500.columns = columns
sp500.loc[sp500['date'].isnull(), 'date'] = '1957-01-01'

# One date is in the wrong format. Correcting it.
sp500.loc[~sp500['date'].str.fullmatch(r'\d{4}-\d{2}-\d{2}'), 'date'] = '1985-01-01'
sp500.loc[:,'date'].apply(lambda x: datetime.strptime(x,'%Y-%m-%d'))
sp500 = pd.melt(sp500, id_vars=['date', 'name', 'cik'], value_vars=['added_ticker'])
sp500


,date,name,cik,variable,value
0,1964-03-31,Abbott Laboratories,1800,added_ticker,ABT
1,2012-12-31,AbbVie Inc.,1551152,added_ticker,ABBV
2,2018-05-31,ABIOMED Inc,815094,added_ticker,ABMD
3,2011-07-06,Accenture plc,1467373,added_ticker,ACN
4,2015-08-31,Activision Blizzard,718877,added_ticker,ATVI
...,...,...,...,...,...
499,1997-10-06,Yum! Brands Inc,1041061,added_ticker,YUM
500,2019-12-23,Zebra Technologies,877212,added_ticker,ZBRA
501,2001-08-07,Zimmer Biomet Holdings,1136869,added_ticker,ZBH
502,2001-06-22,Zions Bancorp,109380,added_ticker,ZION


In [66]:
# Get S&P500 adjustments table and set columns
sp500_adjustments = data[1]
# sp500_adjustments = sp500_adjustments[2:].copy()
columns = ['date', 'added_ticker', 'added_name', 'removed_ticker', 'removed_name', 'reason']
sp500_adjustments.columns = columns
updates = sp500_adjustments[~sp500_adjustments['date'].str.contains(',')].T.shift(1).T
sp500_adjustments['date'].loc[~sp500_adjustments['date'].str.contains(',')] = np.nan
sp500_adjustments[sp500_adjustments['added_ticker'].isnull()]
sp500_adjustments.update(updates)
sp500_adjustments['date'].loc[sp500_adjustments['date'].isnull()] = sp500_adjustments['date'].T.shift(1).T
sp500_adjustments['date'].loc[sp500_adjustments['date'].isnull()] = sp500_adjustments['date'].T.shift(1).T
sp500_adjustments['date'].loc[sp500_adjustments['date'].isnull()] = sp500_adjustments['date'].T.shift(1).T
sp500_adjustments['date'].loc[sp500_adjustments['date'].isnull()] = sp500_adjustments['date'].T.shift(1).T
sp500_adjustments['date'].loc[sp500_adjustments['date'].isnull()] = sp500_adjustments['date'].T.shift(1).T
sp500_additions = sp500_adjustments[~sp500_adjustments['added_ticker'].isnull()]
sp500_additions = sp500_additions[['date', 'added_ticker', 'added_name']]
sp500_additions.rename(columns={'added_name': 'name'}, inplace=True)
sp500_additions = pd.melt(sp500_additions, id_vars=['date','name'], value_vars=['added_ticker'])
sp500_deletions = sp500_adjustments[~sp500_adjustments['removed_ticker'].isnull()]
sp500_deletions = sp500_deletions[['date', 'removed_ticker', 'removed_name']]
sp500_deletions.rename(columns={'removed_name': 'name'}, inplace=True)
sp500_deletions = pd.melt(sp500_deletions, id_vars=['date','name'], value_vars=['removed_ticker'])
sp500_history = pd.concat([sp500_deletions, sp500_additions])

# 将日期转换为 "%Y-%m-%d" 的格式
sp500_history['date'] = sp500_history.loc[:, 'date'].apply(lambda x: datetime.strptime(datetime.strptime(x,'%B %d, %Y').strftime('%Y-%m-%d'), '%Y-%m-%d'))
sp500_history

,date,name,variable,value
0,2020-09-21,H&R Block,removed_ticker,HRB
1,2020-09-21,Coty Inc.,removed_ticker,COTY
2,2020-09-21,Kohl's Corp.,removed_ticker,KSS
3,2020-06-22,Alliance Data Systems,removed_ticker,ADS
4,2020-06-22,Harley-Davidson,removed_ticker,HOG
...,...,...,...,...
248,2000-12-05,Symbol Technologies,added_ticker,SBL
249,2000-12-05,Allegheny Energy,added_ticker,AYE
250,2000-12-05,Ambac Financial,added_ticker,ABK
251,2000-07-27,JDS Uniphase,added_ticker,JDSU


In [68]:
df = pd.concat([sp500, sp500_history], ignore_index=True)
df['date'] = pd.to_datetime(df['date'])
df.sort_values(by='cik', ascending=False, inplace=True)
deduped_df = df[~df.duplicated(['date', 'variable', 'value'])].copy()
deduped_df.sort_values(by='date',inplace=True)
deduped_df.to_csv("sp500_history.csv", index = False)
print(deduped_df.head())

          date                   name        cik      variable value
239 1957-01-01  Huntington Bancshares    49196.0  added_ticker  HBAN
405 1957-01-01       S&P Global, Inc.    64040.0  added_ticker  SPGI
162 1957-01-01              eBay Inc.  1065088.0  added_ticker  EBAY
465 1957-01-01          Valero Energy  1035002.0  added_ticker   VLO
306 1957-01-01        McCormick & Co.    63754.0  added_ticker   MKC


# Constituent File of S&P500 from 1990 to 2018

In [ ]:
import pandas as pd
from datetime import datetime
import numpy as np
import calendar

In [35]:
start = '1957-01-01'
end = '2018-12-31'
sp500_history = pd.read_csv('./sp500_history.csv')
sub_array = np.logical_and(sp500_history['date'] >= start, sp500_history['date'] <= end)
sp500_history = sp500_history.loc[sub_array, :]
sp500_history

,date,name,cik,variable,value
0,1957-01-01,Huntington Bancshares,49196.0,added_ticker,HBAN
1,1957-01-01,"S&P Global, Inc.",64040.0,added_ticker,SPGI
2,1957-01-01,eBay Inc.,1065088.0,added_ticker,EBAY
3,1957-01-01,Valero Energy,1035002.0,added_ticker,VLO
4,1957-01-01,McCormick & Co.,63754.0,added_ticker,MKC
...,...,...,...,...,...
793,2018-12-03,Aetna Inc,NaN,removed_ticker,AET
794,2018-12-03,Rockwell Collins,NaN,removed_ticker,COL
795,2018-12-03,Maxim Integrated Products Inc,743316.0,added_ticker,MXIM
796,2018-12-24,Celanese,1306830.0,added_ticker,CE


In [36]:
# 生成指定日期范围内的日期列表
from dateutil.relativedelta import relativedelta
t1 = datetime.strptime('1957-01', '%Y-%m')
t2 = datetime.strptime('2018-12', '%Y-%m')
monthList = []
while(t1 <= t2):
    monthList.append(t1)
    t1 += relativedelta(months = 1)

In [37]:
SPXconst_dict = {}
stocks = []
for mon in monthList:
    # 获取当前月份的第一天日期和最后一天日期
    monStart = datetime(mon.year, mon.month, 1).strftime(('%Y-%m-%d'))
    monEnd = datetime(mon.year, mon.month, calendar.monthrange(mon.year, mon.month)[1]).strftime(('%Y-%m-%d'))
    # 获取日期范围内的股票信息
    sub_array = np.logical_and(sp500_history['date'] >= monStart, sp500_history['date'] <= monEnd)
    sp500_now = sp500_history.loc[sub_array, :]
    sp500_history = sp500_history.loc[~sub_array, :]
    # 获取新加入和移除的股票名称
    sub_added = np.array(sp500_now['variable'] == 'added_ticker')
    sub_removed = np.array(sp500_now['variable'] == 'removed_ticker')
    stocks_added = sp500_now.loc[sub_added, :]['value'].tolist()
    stocks_removed = sp500_now.loc[sub_removed, :]['value'].tolist()

    # 将新加入的股票放入stocks中
    for stock_add in stocks_added:
        if stock_add not in stocks:
            stocks.append(stock_add)

    # 移除相应股票
    for stock_remove in stocks_removed:
        if stock_remove in stocks:
            stocks.remove(stock_remove)
    

    # 将有效范围日期的数据放入字典
    if(monStart >= '1990-01-01' and monStart <= '2018-12-31'):
        mon = mon.strftime("%m/%Y")
        # 此处要新生成一个list或者np.array，否则对应的value值都会指向最后生成的stocks
        SPXconst_dict[mon] = np.array(stocks)



In [38]:
SPXconst = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in SPXconst_dict.items()]))
SPXconst.to_csv('SPXconst.csv', index=False)

# S&P500 Price Data

Using the scraped list of S&P500 firms, I downloaded the price data for each firms into 505 csv files with the Quandl api. [Reference](https://github.com/imhgchoi/ARIMA-LSTM-hybrid-corrcoef-predict/blob/master/Data_Scraping_Codes.ipynb)

In [1]:
import quandl
import os
import pandas as pd
from datetime import datetime
import numpy as np
import calendar

In [3]:
file_exceptions = "exceptions.txt"
tickers = pd.read_csv('SPXconst.csv')
tickers

,01/1990,02/1990,03/1990,04/1990,05/1990,06/1990,07/1990,08/1990,09/1990,10/1990,...,03/2018,04/2018,05/2018,06/2018,07/2018,08/2018,09/2018,10/2018,11/2018,12/2018
0,HBAN,HBAN,HBAN,HBAN,HBAN,HBAN,HBAN,HBAN,HBAN,HBAN,...,HBAN,HBAN,HBAN,HBAN,HBAN,HBAN,HBAN,HBAN,HBAN,HBAN
1,SPGI,SPGI,SPGI,SPGI,SPGI,SPGI,SPGI,SPGI,SPGI,SPGI,...,SPGI,SPGI,SPGI,SPGI,SPGI,SPGI,SPGI,SPGI,SPGI,SPGI
2,EBAY,EBAY,EBAY,EBAY,EBAY,EBAY,EBAY,EBAY,EBAY,EBAY,...,EBAY,EBAY,EBAY,EBAY,EBAY,EBAY,EBAY,EBAY,EBAY,EBAY
3,VLO,VLO,VLO,VLO,VLO,VLO,VLO,VLO,VLO,VLO,...,VLO,VLO,VLO,VLO,VLO,VLO,VLO,VLO,VLO,VLO
4,MKC,MKC,MKC,MKC,MKC,MKC,MKC,MKC,MKC,MKC,...,MKC,MKC,MKC,MKC,MKC,MKC,MKC,MKC,MKC,MKC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
488,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,KEYS,JKHY
489,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,JKHY,FANG
490,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LW
491,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MXIM


In [18]:
data_open1, data_close1 = getInfo("HBAN", "1999-01-01", "1999-01-25")
data_open2, data_close2 = getInfo("EBAY", "1999-01-01", "1999-01-30")
data_open3, data_close3 = getInfo("EBAY", "1999-02-01", "1999-02-11")

In [19]:
a = pd.concat([data_open1, data_open2], axis=1)
a

,HBAN,EBAY
Date,,
1999-01-04,30.06,236.4
1999-01-05,29.94,228.1
1999-01-06,30.81,234.0
1999-01-07,31.44,272.1
1999-01-08,30.50,315.0
1999-01-11,31.13,304.1
1999-01-12,31.50,280.0
1999-01-13,31.00,218.0
1999-01-14,31.50,224.5


In [22]:
b = pd.concat([a, pd.DataFrame(data_open3)])
b

,HBAN,EBAY
Date,,
1999-01-04,30.06,236.4
1999-01-05,29.94,228.1
1999-01-06,30.81,234.0
1999-01-07,31.44,272.1
1999-01-08,30.50,315.0
1999-01-11,31.13,304.1
1999-01-12,31.50,280.0
1999-01-13,31.00,218.0
1999-01-14,31.50,224.5


In [23]:
b.to_csv('b.csv')

In [28]:
c = pd.DataFrame()
c = pd.concat([c, b], join='outer', axis=0)
c.to_csv('c.csv')

In [7]:
API_KEY = '95NJZYJYyXToregziQw8'

# 获取股票的开盘价和收尾价
def getInfo(ticker, start_date, end_date):
    data = quandl.get("WIKI/{}".format(ticker.replace(".","_")), start_date=start_date, end_date=end_date, api_key=API_KEY)
    data = data.loc[:, ['Open', 'Close']]

    return data['Open'].rename(ticker, inplace=True), data['Close'].rename(ticker, inplace=True)

In [5]:
def main():
    years = [str(x) for x in range(1999, 2019)]

    for year in years:
        tickers_open = pd.DataFrame()
        tickers_close = pd.DataFrame()

        months = [("{:02d}".format(x) + '/' + year) for x in range(1, 13)]
        for month in months:
            # 获取日期信息，以及所有的股票名称
            month_str = month
            month = datetime.strptime(month, "%m/%Y") # 转换成时间戳
            monthFirstDay = datetime(month.year, month.month, 1).strftime(('%Y-%m-%d'))
            monthLastDay = datetime(month.year, month.month, calendar.monthrange(month.year, month.month)[1]).strftime(('%Y-%m-%d'))
            tickers_month =tickers[month_str].dropna().tolist()

            # 获得每个月每支股票的信息， index为日期， cols为股票名称
            month_open = pd.DataFrame()
            month_close = pd.DataFrame()
            for ticker in tickers_month:
                try:
                    data_open, data_close = getInfo(ticker, monthFirstDay, monthLastDay)
                    # 补NaN
                    month_open = pd.concat([month_open, data_open], join='outer', axis=1)
                    month_close = pd.concat([month_close, data_close], join='outer', axis=1)
                except:
                    with open(file_exceptions, 'a') as f:
                        f.write("{} in {}\n".format(ticker, month_str))
                    # print("{} in {}".format(ticker, month_str))
            print("Infomation in {} finished!".format(month_str))
            # 将一年中所有月份的股票信息合并， 空缺处为NaN 
            tickers_open = pd.concat([tickers_open, month_open])
            tickers_close = pd.concat([tickers_close, month_close])

        # 存储每年的股票信息
        tickers_open.to_csv("Open-{}.csv".format(year))
        tickers_close.to_csv("Close-{}.csv".format(year))
        print("S&P500 price data in {} finished!".format(year))


In [7]:
if __name__:
    main()